#Importacion de librerias

In [10]:
import os
import re
import nltk
import numpy as np
from collections import Counter
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils import to_categorical

#Clonacion del repositorio

In [ ]:
!git clone https://github.com/LeydLayd/Analisis-de-sentimientos.git


#Entrenamiento del modelo

In [ ]:

# Descargar stopwords de NLTK
nltk.download('stopwords')

# Configuración de los directorios
repo_dir = "Analisis-de-sentimientos/"
neg_dir = os.path.join(repo_dir, "neg")
pos_dir = os.path.join(repo_dir, "pos")

# Cargar stopwords en inglés
stop_words = set(stopwords.words('english'))


def preprocess_text(text):
    # Eliminar signos de puntuación usando expresiones regulares
    text = re.sub(r'[^\w\s]', '', text)

    # Convertir a minúsculas
    text = text.lower()

    # Eliminar palabras que son solo signos de puntuación o que contienen números
    text = ' '.join([word for word in text.split() if word.isalpha()])

    # Eliminar palabras vacías (stopwords)
    text = ' '.join([word for word in text.split() if word not in stop_words])

    # Eliminar términos cortos (menos de 3 caracteres)
    text = ' '.join([word for word in text.split() if len(word) > 2])

    return text


def load_and_process_data(directory):
    documents = []
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r', encoding='latin-1') as file:
            text = file.read()
            text = preprocess_text(text)
            documents.append(text)
    return documents


# Cargar datos de reseñas negativas y positivas
neg_reviews = load_and_process_data(neg_dir)
pos_reviews = load_and_process_data(pos_dir)

# Separar las últimas 100 opiniones para prueba y el resto para entrenamiento
train_neg = neg_reviews[:-100]
test_neg = neg_reviews[-100:]
train_pos = pos_reviews[:-100]
test_pos = pos_reviews[-100:]

# Unir datos de entrenamiento y prueba
train_docs = train_neg + train_pos
test_docs = test_neg + test_pos

# Etiquetas: 0 para negativo, 1 para positivo
train_labels = [0] * len(train_neg) + [1] * len(train_pos)
test_labels = [0] * len(test_neg) + [1] * len(test_pos)

# Crear el Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_docs)

# Convertir textos a TF-IDF
X_train = tokenizer.texts_to_matrix(train_docs, mode='tfidf')
X_test = tokenizer.texts_to_matrix(test_docs, mode='tfidf')

# Convertir las etiquetas en formato categórico (para clasificación)
y_train = np.array(train_labels)
y_test = np.array(test_labels)

#modelo secuencial
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)), #128 es la red neuronal el inpu_shape es la entradas que es todo el vocabulario
    Dense(64, activation='relu'), # son 3 capaz
    Dense (32, activation = 'relu'),
    Dense(1, activation='sigmoid')  # Para clasificación binaria sigmoid es para hacerlo dos clases
])

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Predicciones
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

# Métricas de evaluación
print("\nPrecisión:", accuracy_score(y_test, y_pred))
print("\nReporte de clasificación:\n", classification_report(y_test, y_pred, target_names=["Negativo", "Positivo"]))

# Calcular y mostrar la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nMatriz de Confusión:\n", conf_matrix)


#Pruebas

In [ ]:
# Crear una función para predecir el sentimiento de un texto
def predict_sentiment(text, model, tokenizer):
    # Preprocesar el texto
    processed_text = preprocess_text(text)

    # Convertir el texto a la representación TF-IDF
    text_tfidf = tokenizer.texts_to_matrix([processed_text], mode='tfidf')

    # Realizar la predicción
    prediction = model.predict(text_tfidf)[0][0]

    # Interpretar el resultado
    sentiment = "Positivo" if prediction > 0.5 else "Negativo"
    confidence = prediction if prediction > 0.5 else 1 - prediction

    return sentiment, confidence

from google.colab import files
from ipywidgets import widgets
from IPython.display import display
import io

# Crear un widget para cargar archivos
upload_button = widgets.Button(
    description='Cargar archivo .txt',
    button_style='info',
    tooltip='Haz clic para cargar un archivo de texto'
)

analyze_button = widgets.Button(
    description='Analizar Sentimiento',
    button_style='primary',
    tooltip='Haz clic para analizar el sentimiento del texto',
    disabled=True  # Inicialmente deshabilitado hasta que se cargue un archivo
)

output = widgets.Output()
file_content = widgets.Textarea(
    value='',
    placeholder='El contenido del archivo se mostrará aquí...',
    description='Contenido:',
    disabled=False,
    layout={'width': '100%', 'height': '200px'}
)

file_info = widgets.HTML(value="<b>Ningún archivo cargado</b>")
uploaded_content = None

def on_upload_clicked(b):
    global uploaded_content

    with output:
        output.clear_output()
        print("Selecciona un archivo .txt de tu computadora...")

        try:
            uploaded = files.upload()

            if uploaded:
                file_name = list(uploaded.keys())[0]
                if not file_name.endswith('.txt'):
                    print(f"Advertencia: El archivo '{file_name}' no tiene extensión .txt, pero intentaremos procesarlo.")

                content = uploaded[file_name]
                try:
                    # Intentar decodificar como UTF-8
                    text_content = content.decode('utf-8')
                except UnicodeDecodeError:
                    # Si falla, intentar con latin-1
                    try:
                        text_content = content.decode('latin-1')
                    except:
                        text_content = content.decode('utf-8', errors='replace')

                file_content.value = text_content
                uploaded_content = text_content
                file_info.value = f"<b>Archivo cargado:</b> {file_name} ({len(text_content)} caracteres)"
                analyze_button.disabled = False
                print(f"Archivo '{file_name}' cargado correctamente. Haz clic en 'Analizar Sentimiento'.")
            else:
                print("No se seleccionó ningún archivo.")
        except Exception as e:
            print(f"Error al cargar el archivo: {str(e)}")

def on_analyze_clicked(b):
    global uploaded_content

    with output:
        output.clear_output()
        if not uploaded_content or uploaded_content.strip() == '':
            print("No hay contenido para analizar. Por favor, carga un archivo primero.")
            return

        print("Analizando sentimiento del archivo...")
        sentiment, confidence = predict_sentiment(uploaded_content, model, tokenizer)
        print(f"Sentimiento: {sentiment}")
        print(f"Confianza: {confidence:.2f}")

        # Análisis adicional - palabras más relevantes
        words = uploaded_content.lower().split()
        preprocessed_words = [word for word in words if word.isalpha() and word not in stop_words and len(word) > 2]

        print(f"\nTotal de palabras procesadas: {len(preprocessed_words)}")
        print("\nMuestra de palabras clave analizadas (primeras 20):")
        print(", ".join(preprocessed_words[:20]))

        # Estadísticas adicionales
        word_count = len(uploaded_content.split())
        char_count = len(uploaded_content)
        print(f"\nEstadísticas del texto: {word_count} palabras, {char_count} caracteres")

upload_button.on_click(on_upload_clicked)
analyze_button.on_click(on_analyze_clicked)

# Mostrar los widgets
print("Análisis de sentimiento de archivos de texto")
display(upload_button)
display(file_info)
display(file_content)
display(analyze_button)
display(output)